In [19]:
import json

json_file_path = '/app/Law_corpus/law-corpus.json'

with open(json_file_path, 'rt') as f_in:
    docs_raw = json.load(f_in)
# docs_raw = [docs_raw]
docs_raw

documents = []

for law_title, law_data in docs_raw.items():
    law_title = law_data.get("law_title")
    law_number = law_data.get("law_number")
    chapters = law_data.get("chapters", [])
    
    for chapter in chapters:
        chapter_title = chapter.get("chapter_title")
        articles = chapter.get("articles", [])
        
        for article in articles:
            doc = {
                "law_title": law_title,
                "law_number": law_number,
                "chapter_title": chapter_title,
                "article_number": article.get("article_number"),
                "title": article.get("title"),
                "content": article.get("content")
            }
            documents.append(doc)

In [7]:
documents[0]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. '}

In [8]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['law_title']}-{doc['chapter_title']}-{doc['title']}-{doc['content'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [9]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [10]:
documents[3]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 4',
 'title': 'Phát triển giáo dục',
 'content': '1. Phát triển giáo dục là quốc sách hàng đầu. 2. Phát triển giáo dục phải gắn với nhu cầu phát triển kinh tế - xã hội, tiến bộ khoa học, công nghệ, củng cố quốc phòng, an ninh; thực hiện chuẩn hóa, hiện đại hóa, xã hội hóa; bảo đảm cân đối cơ cấu ngành nghề, trình độ, nguồn nhân lực và phù hợp vùng miền; mở rộng quy mô trên cơ sở bảo đảm chất lượng và hiệu quả; kết hợp giữa đào tạo và sử dụng. 3. Phát triển hệ thống giáo dục mở, xây dựng xã hội học tập nhằm tạo cơ hội để mọi người được tiếp cận giáo dục, được học tập ở mọi trình độ, mọi hình thức, học tập suốt đời. ',
 'id': '06ce8ae6'}

In [11]:
from collections import defaultdict

In [12]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [13]:
len(hashes), len(documents)

(314, 314)

In [14]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [15]:
hashes['593f7569']

[]

In [16]:
import json
import os

In [17]:
output_dir = "./Law_corpus"
output_path = os.path.join(output_dir, 'documents-with-ids.json')

with open(output_path, 'wt') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=4)

In [12]:
!head ./Law_corpus/documents-with-ids.json

[
    {
        "law_title": "Luật Giáo Dục",
        "law_number": "Luật số: 43/2019/QH14",
        "chapter_title": "Chương I",
        "article_number": "Điều 1",
        "title": "Phạm vi điều chỉnh",
        "content": "Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ",
        "id": "75a9286e"
    },


In [1]:
import json

with open('/app/Law_corpus/documents-with-ids.json', 'rt') as f_in:  # Use /app, not the Windows path
    documents = json.load(f_in)

In [2]:
documents[0]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ',
 'id': '75a9286e'}

In [19]:
prompt_template = """
Bạn mô phỏng một người dùng đang tìm kiếm trợ lý luật.
Dựa trên hồ sơ Luật Việt Nam được cung cấp, hãy tạo ra **1 cặp câu hỏi và câu trả lời** (question-answer pair).
Câu hỏi phải là câu hỏi mà một người dân bình thường có thể đặt ra khi tìm hiểu về luật này, liên quan trực tiếp đến nội dung của hồ sơ luật.
Câu trả lời phải được trích xuất từ thông tin trong hồ sơ, vừa đủ, không dài không ngắn, và chính xác.

Hồ sơ luật:

law_title: {law_title}
title: {title}
content: {content}

Cung cấp đầu ra dưới dạng JSON có thể phân tích được mà không sử dụng các khối mã:

{{
  "generated_question": "Câu hỏi mà người dân có thể đặt ra khi tìm hiểu về luật này",
  "generated_answer": "Câu trả lời được trích xuất từ hồ sơ luật"
}}
""".strip()

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")
client = OpenAI(
    api_key = DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com"
)

In [10]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    json_response = response.choices[0].message.content
    return json_response

In [11]:
from tqdm.auto import tqdm

In [12]:
results = {}

## Generate 100 question-answer pair

In [20]:
len(documents)

314

In [31]:
import random
# Randomly sample 100 documents
sampled_documents = random.sample(documents, 150)

In [33]:
sampled_documents[[id]=='0f1d84b3']

{'law_title': 'Luật Giao Thông Đường Bộ',
 'law_number': 'Luật số: 23/2008/QH12',
 'chapter_title': 'CHƯƠNG II',
 'article_number': 'Điều 10',
 'title': 'Hệ thống báo hiệu đường bộ',
 'content': '1. Hệ thống báo hiệu đường bộ gồm hiệu lệnh của người điều khiển giao thông; tín hiệu đèn giao thông, biển báo hiệu, vạch kẻ đường, cọc tiêu hoặc tường bảo vệ, rào chắn. 2. Hiệu lệnh của người điều khiển giao thông quy định như sau: a) Tay giơ thẳng đứng để báo hiệu cho người tham gia giao thông ở các hướng dừng lại; b) Hai tay hoặc một tay dang ngang để báo hiệu cho người tham gia giao thông ở phía trước và ở phía sau người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía bên phải và bên trái của người điều khiển giao thông được đi; c) Tay phải giơ về phía trước để báo hiệu cho người tham gia giao thông ở phía sau và bên phải người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía trước người điều khiển giao thông được rẽ phải; người tham gia giao thôn

In [ ]:
results = []

# Process each sampled document
for doc in tqdm(sampled_documents):
    doc_id = doc['id']
    
    # Generate question-answer pair using the API
    qa_pair_json = generate_questions(doc)
    
    # Parse JSON response and add document ID to the result
    qa_pair = json.loads(qa_pair_json)
    qa_pair['doc_id'] = doc_id
    
    # Append to results
    results.append(qa_pair)

In [37]:
results

[{'generated_question': 'Khi gặp tín hiệu đèn vàng nhấp nháy, người tham gia giao thông cần làm gì?',
  'generated_answer': 'Khi gặp tín hiệu đèn vàng nhấp nháy, người tham gia giao thông được đi nhưng phải giảm tốc độ, chú ý quan sát và nhường đường cho người đi bộ qua đường.',
  'doc_id': '0e6a08af'},
 {'generated_question': 'Các loại hình cơ sở giáo dục mầm non được quy định như thế nào trong Luật Giáo Dục?',
  'generated_answer': 'Cơ sở giáo dục mầm non bao gồm: 1. Nhà trẻ, nhóm trẻ độc lập nhận trẻ em từ 03 tháng tuổi đến 03 tuổi; 2. Trường mẫu giáo, lớp mẫu giáo độc lập nhận trẻ em từ 03 tuổi đến 06 tuổi; 3. Trường mầm non, lớp mầm non độc lập là cơ sở giáo dục kết hợp nhà trẻ và mẫu giáo, nhận trẻ em từ 03 tháng tuổi đến 06 tuổi.',
  'doc_id': 'a0fcdc5b'},
 {'generated_question': 'Nếu tôi là công dân Việt Nam kết hôn với người nước ngoài và muốn ly hôn, tôi cần làm thủ tục ở đâu?',
  'generated_answer': 'Việc ly hôn giữa công dân Việt Nam với người nước ngoài, giữa người nước ng

In [1]:
import pandas as pd

# Convert results to a DataFrame
df = pd.DataFrame(results)

# Save to CSV
df.to_csv('ground-truth-data.csv', index=False, encoding='utf-8')

NameError: name 'results' is not defined

## Add law_title to groundtruth dataset

In [4]:
import pandas as pd

df_ground_truth = pd.read_csv('./evaluate/ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [6]:
df_ground_truth

,generated_question,generated_answer,doc_id
0,Các loại hình cơ sở giáo dục mầm non được quy ...,"Cơ sở giáo dục mầm non bao gồm: 1. Nhà trẻ, nh...",a0fcdc5b
1,Nếu tôi là công dân Việt Nam kết hôn với người...,Việc ly hôn giữa công dân Việt Nam với người n...,a2a23d08
2,"Khi nào cô, dì, chú, cậu, bác ruột có nghĩa vụ...","Cô, dì, chú, cậu, bác ruột không sống chung vớ...",36501b98
3,Tại nơi có cả biển báo hiệu cố định và báo hiệ...,Tại nơi có biển báo hiệu cố định lại có báo hi...,4d5249af
4,Khi nào tôi được phép vượt xe bên phải thay vì...,Bạn được phép vượt xe bên phải trong các trườn...,7d030583
...,...,...,...
96,Luật Hôn Nhân Và Gia Đình điều chỉnh những vấn...,Luật này quy định chế độ hôn nhân và gia đình;...,424554a8
97,Điều kiện để một cặp vợ chồng được nhờ người m...,Vợ chồng có quyền nhờ người mang thai hộ khi c...,04868edf
98,Cha mẹ có quyền tự mình thực hiện giao dịch tà...,Cha hoặc mẹ có quyền tự mình thực hiện giao dị...,dc292a9c
99,Vợ chồng có nghĩa vụ gì đối với quyền tự do tí...,"Vợ, chồng có nghĩa vụ tôn trọng quyền tự do tí...",c8a7e8e1


In [49]:
!head ground-truth-data.csv

generated_question                                                                                                                                                               ,generated_answer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ,doc_id
Các loại hình cơ sở giáo dục mầm non được quy định như thế nào trong Luật Giáo Dục?                                                           

In [7]:
ground_truth[0]

{'generated_question': 'Các loại hình cơ sở giáo dục mầm non được quy định như thế nào trong Luật Giáo Dục?',
 'generated_answer': 'Cơ sở giáo dục mầm non bao gồm: 1. Nhà trẻ, nhóm trẻ độc lập nhận trẻ em từ 03 tháng tuổi đến 03 tuổi; 2. Trường mẫu giáo, lớp mẫu giáo độc lập nhận trẻ em từ 03 tuổi đến 06 tuổi; 3. Trường mầm non, lớp mầm non độc lập là cơ sở giáo dục kết hợp nhà trẻ và mẫu giáo, nhận trẻ em từ 03 tháng tuổi đến 06 tuổi.',
 'doc_id': 'a0fcdc5b'}

In [8]:
import json

with open('/app/Law_corpus/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [13]:
documents[0] 

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 1',
 'title': 'Phạm vi điều chỉnh',
 'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ',
 'id': '75a9286e'}

In [16]:
def find_document_by_id(doc_id, documents):
    for doc in documents:
        if doc['id'] == doc_id:
            return doc
    return None

# Update the ground_truth list directly
for item in ground_truth:
    doc_id = item['doc_id']
    doc = find_document_by_id(doc_id, documents)
    if doc:
        item['law_title'] = doc.get('law_title', 'Unknown Law Title')

# Convert the updated list back to a DataFrame
df_updated = pd.DataFrame(ground_truth)

In [18]:
ground_truth[-1]

{'generated_question': 'Tài sản nào được coi là tài sản riêng của vợ hoặc chồng theo Luật Hôn Nhân Và Gia Đình?',
 'generated_answer': 'Tài sản riêng của vợ, chồng gồm tài sản mà mỗi người có trước khi kết hôn; tài sản được thừa kế riêng, được tặng cho riêng trong thời kỳ hôn nhân; tài sản được chia riêng cho vợ, chồng theo quy định tại các điều 38, 39 và 40 của Luật này; tài sản phục vụ nhu cầu thiết yếu của vợ, chồng và tài sản khác mà theo quy định của pháp luật thuộc sở hữu riêng của vợ, chồng.',
 'doc_id': '48bc6ff9',
 'law_title': 'Luật Hôn Nhân Và Gia Đình'}

In [19]:
df_updated

,generated_question,generated_answer,doc_id,law_title
0,Các loại hình cơ sở giáo dục mầm non được quy ...,"Cơ sở giáo dục mầm non bao gồm: 1. Nhà trẻ, nh...",a0fcdc5b,Luật Giáo Dục
1,Nếu tôi là công dân Việt Nam kết hôn với người...,Việc ly hôn giữa công dân Việt Nam với người n...,a2a23d08,Luật Hôn Nhân Và Gia Đình
2,"Khi nào cô, dì, chú, cậu, bác ruột có nghĩa vụ...","Cô, dì, chú, cậu, bác ruột không sống chung vớ...",36501b98,Luật Hôn Nhân Và Gia Đình
3,Tại nơi có cả biển báo hiệu cố định và báo hiệ...,Tại nơi có biển báo hiệu cố định lại có báo hi...,4d5249af,Luật Giao Thông Đường Bộ
4,Khi nào tôi được phép vượt xe bên phải thay vì...,Bạn được phép vượt xe bên phải trong các trườn...,7d030583,Luật Giao Thông Đường Bộ
...,...,...,...,...
96,Luật Hôn Nhân Và Gia Đình điều chỉnh những vấn...,Luật này quy định chế độ hôn nhân và gia đình;...,424554a8,Luật Hôn Nhân Và Gia Đình
97,Điều kiện để một cặp vợ chồng được nhờ người m...,Vợ chồng có quyền nhờ người mang thai hộ khi c...,04868edf,Luật Hôn Nhân Và Gia Đình
98,Cha mẹ có quyền tự mình thực hiện giao dịch tà...,Cha hoặc mẹ có quyền tự mình thực hiện giao dị...,dc292a9c,Luật Hôn Nhân Và Gia Đình
99,Vợ chồng có nghĩa vụ gì đối với quyền tự do tí...,"Vợ, chồng có nghĩa vụ tôn trọng quyền tự do tí...",c8a7e8e1,Luật Hôn Nhân Và Gia Đình


In [20]:
# Strip extra spaces from string values in the DataFrame
df_updated['generated_question'] = df_updated['generated_question'].str.strip()
df_updated['generated_answer'] = df_updated['generated_answer'].str.strip()

# Save the updated DataFrame to CSV
df_updated.to_csv('/app/evaluate/ground-truth-data.csv', index=False, encoding='utf-8')